## Vistual inertia scheduling

In [1]:
import andes
import os

from statistics import fmean
from andes.interop.pandapower import to_pandapower
from andes.interop.pandapower import make_GSF, build_group_table
import gurobipy as gb
import pandas as pd
import numpy as np
import logging

logger = logging.getLogger(__name__)

In [2]:
from opf import dcopf # base class

'''
    Vittual Inertia Scheduling (vis) solves economic dispatch problem
    consiering the dynamic frequency constriants and vsg invertia 
    support reserve.

    The base class is inherited from 'dcopf' in opf.py

    1) vis1: dcopf + fnadir/RoCof (ML linearization)

    2) vis2: dcopf + fnadir/RoCof (ML linearization)
                   + VSG power reserve (ML linearization)

    3) vis3: dcopf + fnadir/RoCof (ML linearization)
                   + VSG power reserve (Final value theorem)
'''

'''
    TODO: 1) write f, as constriants
          2) wirte power reserve constriants
'''

# ---------------------------------------------------------

class vis1(dcopf):
    """
    vis1: dcopf + fnadir/RoCof (ML linearization)
    """

    def __init__(self, name='vis1', norm=None, nn=None, nn_num=64, dpe=0.01):
        """
        Initialize high level parameters

        Input
        ---------
        name: str
            name
        norm: dict
            normalizatin parameter for fnadir and Pvsg prediction
        nn: dict
            neural network weight and bias
        nn_num: integer
            number of MLP nuerols (assume single layer MLP)
        dpe: float
            delta Pe, power mismatch, or load change
        """
        super().__init__(name)
        self.norm = norm 
        self.nn = nn
        self.nn_num = nn_num
        self.fnadir = 0.01 # TODO 0.6Hz
        self.rocof = 0.01 # TODO 0.6Hz/s
        self.dpe = dpe

    def from_andes(self, ssa, typeII=None):
        """
        Initialize parameters from andes mdoel

        Input
        ---------
        ssa: andes model
        typeII: list
            idx of typeII generator (vsg inverter), i.g., ['PV_6', 'PV_7']

        """      
        super().from_andes(ssa)

        # Define typeII generatiors, defalt typeI (type=1)
        self.gen['type'] = 1
        if typeII:
            for idx in typeII:
                row = self.gen[self.gen['idx'] == idx].index[0]
                self.gen['type'].iloc[row] = 2

        # add new parameter
        self.gen['p_pre'] = 0
        self.gen['band'] = self.gen['pmax'] - self.gen['pmin']
        self.gen['Sn'] /= self.mva  # normalize Sn
        self.gen['K'] = 1

        # load new parameter from andes
        genrow = ssa.GENROU.as_df()
        regc = ssa.REGCV2.as_df()
        tgov = ssa.TGOV1N.as_df()
        tgov.rename(columns={'idx':'gov', 'syn':'idx'}, inplace=True)
        regc.rename(columns={'idx':'vsg', 'gen':'idx', 'M':'Mvsg', 'D': 'Dvsg'},  inplace=True)
        # merge dataframe
        genrow = pd.merge(left=genrow, right=tgov[['idx', 'R']], on='idx', how='left')
        genrow.rename(columns={'idx': 'syn', 'gen': 'idx'}, inplace=True)
        self.gen = pd.merge(left=self.gen, right=genrow[['idx', 'M','D', 'R']], on='idx', how='left')
        self.gen = pd.merge(left=self.gen, right=regc[['idx', 'Mvsg', 'Dvsg']], on='idx', how='left')
        # fill nan caused by merge
        self.gen.fillna(0, inplace=True)

        self.update_dict()

    def build(self):
        # self.data_check()

        # --- build gurobi model ---
        self.update_dict()
        self.mdl = gb.Model(self.name)
        self._build_vars()
        self._build_obj()
        self._build_cons()
        print('Successfully build vis0 model.')
        logger.info('Successfully build vis0 model.')

    def _build_vars(self):
        GEN = self.gendict.keys()

        # --- uncontrollable generators limit to p0 ---
        gencp = self.gen.copy()
        gencp['pmax'][gencp.ctrl == 0] = gencp['p0'][gencp.ctrl == 0]
        gencp['pmin'][gencp.ctrl == 0] = gencp['p0'][gencp.ctrl == 0]
        # --- offline geenrators limit to 0 ---
        gencp['pmax'][gencp.u == 0] = 0
        gencp['pmin'][gencp.u == 0] = 0

        # --- gen: pg ---
        self.pg = self.mdl.addVars(GEN, name='pg', vtype=gb.GRB.CONTINUOUS, obj=0,
                              ub=gencp.pmax.tolist(), lb=gencp.pmin.tolist())
        # --- RegUp, RegDn --- !!! modify inverter reserve up and down in andes file, prd=0
        self.pru = self.mdl.addVars(GEN, name='pru', vtype=gb.GRB.CONTINUOUS, obj=0,
                               ub=gencp.band.tolist(), lb=[0] * gencp.shape[0])
        self.prd = self.mdl.addVars(GEN, name='prd', vtype=gb.GRB.CONTINUOUS, obj=0,
                               ub=gencp.band.tolist(), lb=[0] * gencp.shape[0])
        
        # --- Mvsg, Dvsg ---
        gendict = self.gendict
        gendict_II = dict()
        for (new_key, new_value) in gendict.items():
            if new_value['type'] == 2:
                gendict_II[new_key] = new_value
        vsg = gendict_II.keys()

        self.Mvsg = self.mdl.addVars(vsg, name='Mvsg', vtype=gb.GRB.CONTINUOUS, obj=0,
                               ub=[4]*len(vsg), lb=[0]*len(vsg))
        self.Dvsg = self.mdl.addVars(vsg, name='Dvsg', vtype=gb.GRB.CONTINUOUS, obj=0,
                               ub=[5]*len(vsg), lb=[0]*len(vsg))

        # print('Successfully build var.')

    def _build_obj(self):
        GEN = self.gendict.keys()
        gendict = self.gendict
        costdict = self.costdict

        # --- minimize generation cost ---
        cost_pg = sum(self.pg[gen] * costdict[gen]['c1']
                      + self.pg[gen] * self.pg[gen] * costdict[gen]['c2']
                      + costdict[gen]['c0'] * gendict[gen]['u']  # online status
                      for gen in GEN)

        # --- RegUp, RegDn cost ---
        cost_ru = sum(self.pru[gen] * costdict[gen]['cru'] for gen in GEN)
        cost_rd = sum(self.prd[gen] * costdict[gen]['crd'] for gen in GEN)
        cost_vsg = cost_ru + cost_rd

        self.obj = self.mdl.setObjective(expr=cost_pg + cost_vsg, sense=gb.GRB.MINIMIZE)
        # print('Successfully build obj.')

    def _build_cons(self):
        # --- var idx ---
        ptotal = self.load.p0.sum()

        gendict = self.gendict
        linedict = self.linedict
        gen_gsfdict = self.gen_gsfdict

        GEN = gendict.keys()
        LINE = linedict.keys()

        # --- 01 power balance ---
        p_sum = sum(self.pg[gen] for gen in GEN)
        self.mdl.addConstr(p_sum == ptotal, name='PowerBalance')

        # --- 02 line limits ---
        for line in LINE:
            lhs1 = sum(self.pg[gen] * gen_gsfdict[gen][line] for gen in GEN)
            self.mdl.addConstr(lhs1+linedict[line]['sup'] <= linedict[line]['rate_a'], name=f'{line}_U')
            self.mdl.addConstr(lhs1+linedict[line]['sup'] >= -linedict[line]['rate_a'], name=f'{line}_D')

        # --- 03 dynamic frequency constraints --
        self._add_fcons()

        #print('Successfully build cons.')

    def _add_fcons(self):

        gendict = self.gendict
        GENI, GENII = self._get_GENI_GENII_key()
        self.GENI_test, self.GENII_test = self._get_GENI_GENII_key()
        
        # --- Synthetic M/D/F/R ---
        Msys = sum(gendict[gen]['Sn'] * gendict[gen]['M'] for gen in GENI)
        Msys += sum(gendict[gen]['Sn'] * self.Mvsg[gen] for gen in GENII)
        Msys /= sum(gendict[gen]['Sn'] for gen in gendict.keys())
        self.Msys = Msys

        Dsys = sum(gendict[gen]['Sn'] * gendict[gen]['D'] for gen in GENI)
        Dsys += sum(gendict[gen]['Sn'] * self.Dvsg[gen] for gen in GENII)
        Dsys /= sum(gendict[gen]['Sn'] for gen in gendict.keys())

        Rsys = sum(gendict[gen]['K'] / gendict[gen]['R'] * gendict[gen]['Sn'] for gen in GENI)
        Rsys /= sum(gendict[gen]['Sn'] for gen in GENI)

        Fsys = sum(gendict[gen]['K'] / gendict[gen]['R'] * self.pg[gen] for gen in GENI)
        Fsys /= sum(gendict[gen]['Sn'] for gen in GENI)       
        
        # --- add gurobi var for fnadir ---
        af = []
        zf = []
        for i in range(self.nn_num):
            af.append('af'+str(i))
            zf.append('zf'+str(i))
        self.af_key = af
        self.zf_key = zf
        self.af = self.mdl.addVars(af, name='af', vtype=gb.GRB.BINARY)
        self.zf = self.mdl.addVars(zf, name='zf', vtype=gb.GRB.CONTINUOUS, obj=0,
                                lb=[0]*self.nn_num)

        # --- add constraints ---
        # --- RoCof ----
        self.mdl.addConstr(self.rocof * Msys >= abs(self.dpe), name='RoCof')

        # --- fnadir ----
        # - self.fnadir <= fnadir <= self.fnadir
        fnorm = self.norm['fnorm']
        Msys_norm = (Msys - fnorm['M'].iloc[0]) / fnorm['M'].iloc[1]
        Dsys_norm = (Dsys - fnorm['D'].iloc[0]) / fnorm['D'].iloc[1]
        Fsys_norm = (Fsys - fnorm['Fg'].iloc[0]) / fnorm['Fg'].iloc[1]
        Rsys_norm = (Rsys - fnorm['Rg'].iloc[0]) / fnorm['Rg'].iloc[1]

        zf_bar = []
        num_idx = []
        for i in range(self.nn_num):
            zf_bar_temp = Msys_norm * self.nn['fw1'][0].iloc[i] + \
                          Dsys_norm * self.nn['fw1'][1].iloc[i] + \
                          Fsys_norm * self.nn['fw1'][2].iloc[i] + \
                          Rsys_norm * self.nn['fw1'][3].iloc[i] + self.nn['fb1'][0].iloc[i]
            zf_bar.append(zf_bar_temp)
            num_idx.append(i)
        
        self.zf_bar = zf_bar

        hdown = -100
        hup = 100

        for zf_key, af_key, zf_bar_ in zip(self.zf_key, self.af_key, zf_bar): # check this zip
            self.mdl.addConstr( self.zf[zf_key] <= zf_bar_ - hdown*(1 - self.af[af_key]) )
            self.mdl.addConstr( self.zf[zf_key] >= zf_bar_ )
            self.mdl.addConstr( self.zf[zf_key] <= hup * self.af[af_key] )

        fnadir_norm = 0.0
        for zf_key, i in zip(self.zf_key, num_idx):
            fnadir_norm += self.zf[zf_key] * self.nn['fw2'][i].iloc[0]

        fnadir_norm += self.nn['fb2'][0].iloc[0]

        fnadir_pred = fnadir_norm * fnorm['fnadir'].iloc[1] + fnorm['fnadir'].iloc[0]
        fnadir_pred *= self.dpe

        self.mdl.addConstr(fnadir_pred >= - self.fnadir, name=f'fnadir_D')
        self.mdl.addConstr(fnadir_pred <= self.fnadir, name=f'fnadir_U')

        print('succefully add frequency constraints')

    def _get_GENI_GENII_key(self):
        gendict = self.gendict

        # --- filter Type II gen ---
        gendict_I, gendict_II= dict(), dict()
        for (new_key, new_value) in gendict.items():
            if new_value['type'] == 1:
                gendict_I[new_key] = new_value
        for (new_key, new_value) in gendict.items():
            if new_value['type'] == 2:
                gendict_II[new_key] = new_value

        return gendict_I.keys(), gendict_II.keys()

    def get_res(self):
        """
        Get resutlts, can be used after mdl.optimize().

        Returns
        -------
        DataFrame
            The output DataFrame contains setpoints ``pg``

        """
        self.build()
        self.mdl.optimize()
        # --- check if mdl is sovled ---
        if not hasattr(self.pg[self.gen.idx[0]], 'X'):
            logger.warning('vis1 has no valid resutls!')
            pg = [0] * self.gen.shape[0]
            pru = [0] * self.gen.shape[0]
            prd = [0] * self.gen.shape[0]
        else:
            logger.warning('Successfully solve vis1.')
            # --- gather data --
            pg = []
            pru = []
            prd = []
            for gen in self.gendict.keys():
                pg.append(self.pg[gen].X)
                pru.append(self.pru[gen].X)
                prd.append(self.prd[gen].X)
            # --- cost ---
            self.res_cost = self.mdl.getObjective().getValue()
            logger.info(f'Total cost={np.round(self.res_cost, 3)}')
            
        # --- build output table ---
        dcres = pd.DataFrame()
        dcres['gen'] = self.gen['idx']
        dcres['pg'] = pg
        dcres['pru'] = pru
        dcres['prd'] = prd
        dcres.fillna(0, inplace=True)
        return dcres

## Main - IEEE 39


### load case from andes excel

In [3]:
# get andes case from excel
dir_path = os.path.abspath('..')
case_path = '/VIS_opf/ieee14_vis.xlsx'
case = dir_path + case_path
ssa = andes.load(case, no_output=True)

REGCV2: unused data {'KpId': 50, 'KiId': 100, 'KpIq': 50, 'KiIq': 100}
REGCV2: unused data {'KpId': 50, 'KiId': 100, 'KpIq': 50, 'KiIq': 100}


### load norm parameter

In [4]:
# prepare nn data for visopf
data_path = dir_path + '/VIS_opf/NN_train'

fnorm = pd.read_csv(data_path + '/fnorm.csv')
pnorm = pd.read_csv(data_path + '/pnorm.csv')
norm = {'fnorm': fnorm, 'pnorm': pnorm }
norm


{'fnorm':            M          D         Fg          Rg    fnadir
 0  19.895678  12.197409  80.138509  126.078088 -0.012637
 1   5.744600   4.344583  39.902459   43.025549  0.005493,
 'pnorm':            M          D         Fg          Rg      Mvsg      Dvsg     Ppeak
 0  19.895678  12.197409  80.138509  126.078088  0.981245  2.533602  0.067277
 1   5.744600   4.344583  39.902459   43.025549  0.587552  0.867664  0.036406}

In [5]:
# example to call norm data
norm['fnorm']['M'].iloc[0] # mean
norm['fnorm']['M'].iloc[1] # std 

5.74459957980676

### load nn parameter

In [6]:
fw1 = pd.read_csv(data_path + '/fw1.csv', header=None)
fw2 = pd.read_csv(data_path + '/fw2.csv', header=None)

fb1 = pd.read_csv(data_path + '/fb1.csv', header=None)
fb2 = pd.read_csv(data_path + '/fb2.csv', header=None)

In [7]:
pw1 = pd.read_csv(data_path + '/pw1.csv', header=None)
pw2 = pd.read_csv(data_path + '/pw2.csv', header=None)

pb1 = pd.read_csv(data_path + '/pb1.csv', header=None)
pb2 = pd.read_csv(data_path + '/pb2.csv', header=None)

In [8]:
nn = {
        'fw1': fw1,
        'fw2': fw2,       
        'fb1': fb1,
        'fb2': fb2,
        'pw1': pw1,
        'pw2': pw2,
        'pb1': pb1,
        'pb2': pb2,
    }

### test opf model

In [9]:
ss = vis1(norm=norm, nn=nn, dpe=0.05)
ss.norm

Restricted license - for non-production use only - expires 2023-10-25


{'fnorm':            M          D         Fg          Rg    fnadir
 0  19.895678  12.197409  80.138509  126.078088 -0.012637
 1   5.744600   4.344583  39.902459   43.025549  0.005493,
 'pnorm':            M          D         Fg          Rg      Mvsg      Dvsg     Ppeak
 0  19.895678  12.197409  80.138509  126.078088  0.981245  2.533602  0.067277
 1   5.744600   4.344583  39.902459   43.025549  0.587552  0.867664  0.036406}

In [10]:
ss.norm['fnorm']['M'].iloc[0]

19.895678

In [11]:
ss.nn['fw1']

,0,1,2,3
0,0.053316,-0.018950,0.103133,0.005192
1,0.090899,-0.021882,-0.052110,-0.066491
2,-0.054552,0.081501,0.133793,0.073621
3,0.013614,0.007082,0.004717,-0.008138
4,0.017724,-0.007474,0.009466,-0.007022
...,...,...,...,...
59,0.009615,-0.005689,-0.002588,-0.018844
60,0.064262,-0.043027,0.112759,-0.003101
61,-0.140001,-0.125340,-0.942938,-0.133099
62,0.006305,-0.006394,-0.003145,0.004949


In [12]:
ss.nn['fw1'][1].iloc[1]
# ss.nn['fb1']

-0.021882491

In [13]:
ss.from_andes(ssa, ['PV_6', 'PV_7'])

In [14]:
ss.gen

,idx,u,name,Sn,Vn,bus,p0,pmax,pmin,v0,...,ramp30,type,p_pre,band,K,M,D,R,Mvsg,Dvsg
0,PV_2,1.0,PV_2,1.0,69.0,2,0.40000,0.5,0.1,1.03,...,600,1,0,0.4,1,8.0,0.0,0.12,0.0,0.0
1,PV_3,1.0,PV_3,1.0,69.0,3,0.40000,0.5,0.1,1.01,...,600,1,0,0.4,1,5.0,0.0,0.12,0.0,0.0
2,PV_4,1.0,PV_4,1.0,138.0,6,0.30000,1.0,0.1,1.03,...,600,1,0,0.9,1,5.0,0.0,0.05,0.0,0.0
3,PV_5,1.0,PV_5,1.0,69.0,8,0.30000,0.5,0.1,1.03,...,600,1,0,0.4,1,10.0,0.0,0.05,0.0,0.0
4,PV_6,1.0,PV_6,1.0,138.0,14,0.10000,0.1,0.0,1.01,...,600,2,0,0.1,1,0.0,0.0,0.00,0.0,3.0
5,PV_7,1.0,PV_7,1.0,138.0,12,0.10000,0.1,0.0,1.01,...,600,2,0,0.1,1,0.0,0.0,0.00,6.0,2.0
6,Slack_1,1.0,Slack_1,1.0,69.0,1,0.81442,3.0,0.5,1.03,...,600,1,0,2.5,1,8.0,0.0,0.12,0.0,0.0


In [15]:
# ss.gendict

In [16]:
ss.norm

{'fnorm':            M          D         Fg          Rg    fnadir
 0  19.895678  12.197409  80.138509  126.078088 -0.012637
 1   5.744600   4.344583  39.902459   43.025549  0.005493,
 'pnorm':            M          D         Fg          Rg      Mvsg      Dvsg     Ppeak
 0  19.895678  12.197409  80.138509  126.078088  0.981245  2.533602  0.067277
 1   5.744600   4.344583  39.902459   43.025549  0.587552  0.867664  0.036406}

In [17]:
a_test = []
for i in range(64):
    a_test.append('a'+str(i))
# a_test

In [18]:
ss.gendict.keys()

dict_keys(['PV_2', 'PV_3', 'PV_4', 'PV_5', 'PV_6', 'PV_7', 'Slack_1'])

In [19]:
ss.nn['fw2'][4].iloc[0]

0.009975906

In [20]:
ss.nn['fb1']

,0
0,0.288764
1,0.193274
2,0.329569
3,-0.045777
4,-0.388495
...,...
59,-0.407754
60,0.345564
61,-1.168843
62,-0.120155


In [21]:
ss.nn['fb2'][0].iloc[0]

-0.09059838

In [22]:
ss.build()

# ss.mdl.optimize()

succefully add frequency constraints
Successfully build vis0 model.


In [23]:
ss.build()

succefully add frequency constraints
Successfully build vis0 model.


In [24]:
ss.get_res()

succefully add frequency constraints
Successfully build vis0 model.
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 228 rows, 153 columns and 1769 nonzeros
Model fingerprint: 0x7ae34f27
Variable types: 89 continuous, 64 integer (64 binary)
Coefficient statistics:
  Matrix range     [2e-07, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-01, 5e+00]
  RHS range        [1e-03, 1e+02]
Found heuristic solution: objective 2.2870000
Presolve removed 220 rows and 142 columns
Presolve time: 0.01s
Presolved: 8 rows, 11 columns, 39 nonzeros
Variable types: 9 continuous, 2 integer (2 binary)

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 2.287 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.287000

Successfully solve vis1.


,gen,pg,pru,prd
0,PV_2,0.100,0.0,0.0
1,PV_3,0.100,0.0,0.0
2,PV_4,0.100,0.0,0.0
3,PV_5,0.100,0.0,0.0
4,PV_6,0.100,0.0,0.0
5,PV_7,0.100,0.0,0.0
6,Slack_1,1.687,0.0,0.0


In [25]:
ss.load.p0.sum()

ss.rocof

ss.Msys # check whether dpe is reasonable

<gurobi.LinExpr: 5.142857142857142 + 0.14285714285714285 Mvsg[PV_6] + 0.14285714285714285 Mvsg[PV_7]>

In [26]:
ss.costdict

{'PV_2': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_3': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_4': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_5': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_6': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_7': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'Slack_1': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0}}

In [27]:
ss.zf


{'zf0': <gurobi.Var zf[zf0] (value 0.0)>,
 'zf1': <gurobi.Var zf[zf1] (value 0.3064117821784213)>,
 'zf2': <gurobi.Var zf[zf2] (value 0.0)>,
 'zf3': <gurobi.Var zf[zf3] (value 0.0)>,
 'zf4': <gurobi.Var zf[zf4] (value 0.0)>,
 'zf5': <gurobi.Var zf[zf5] (value 0.0)>,
 'zf6': <gurobi.Var zf[zf6] (value 0.7402315456098424)>,
 'zf7': <gurobi.Var zf[zf7] (value 0.5408073259134755)>,
 'zf8': <gurobi.Var zf[zf8] (value 0.0)>,
 'zf9': <gurobi.Var zf[zf9] (value 0.7976443471397059)>,
 'zf10': <gurobi.Var zf[zf10] (value 0.5455574127432377)>,
 'zf11': <gurobi.Var zf[zf11] (value 0.7163692902948886)>,
 'zf12': <gurobi.Var zf[zf12] (value 0.0)>,
 'zf13': <gurobi.Var zf[zf13] (value 0.0)>,
 'zf14': <gurobi.Var zf[zf14] (value 0.0)>,
 'zf15': <gurobi.Var zf[zf15] (value 0.48619228348922544)>,
 'zf16': <gurobi.Var zf[zf16] (value 0.4739083432229492)>,
 'zf17': <gurobi.Var zf[zf17] (value 0.03193157890766157)>,
 'zf18': <gurobi.Var zf[zf18] (value 0.0)>,
 'zf19': <gurobi.Var zf[zf19] (value 0.68680319

In [28]:
ss.mdl.display()

Minimize
<gurobi.LinExpr: pg[PV_2] + pg[PV_3] + pg[PV_4] + pg[PV_5] + pg[PV_6] + pg[PV_7]
+ pg[Slack_1]>
Subject To
PowerBalance: <gurobi.LinExpr: pg[PV_2] + pg[PV_3] + pg[PV_4] + pg[PV_5] + pg[PV_6] +
 pg[PV_7] + pg[Slack_1]> = 2.287
Line_1_U: <gurobi.LinExpr: -0.8380258373889509 pg[PV_3] + -0.610558006232354 pg[PV_4] +
-0.6573114828670692 pg[PV_5] + -0.632996511342461 pg[PV_6] + -0.6391136099512122
 pg[PV_7]> <= 0.4592
Line_1_D: <gurobi.LinExpr: -0.8380258373889509 pg[PV_3] + -0.610558006232354 pg[PV_4] +
-0.6573114828670692 pg[PV_5] + -0.632996511342461 pg[PV_6] + -0.6391136099512122
 pg[PV_7]> >= -3.5408
Line_2_U: <gurobi.LinExpr: -0.16197416261104905 pg[PV_3] + -0.3894419937676455 pg[PV_4]
+ -0.3426885171329303 pg[PV_5] + -0.36700348865753935 pg[PV_6] + -0.36088639004878814
 pg[PV_7]> <= 0.3038
Line_2_D: <gurobi.LinExpr: -0.16197416261104905 pg[PV_3] + -0.3894419937676455 pg[PV_4]
+ -0.3426885171329303 pg[PV_5] + -0.36700348865753935 pg[PV_6] + -0.36088639004878814
 pg[PV_7]> >= -

In [29]:
ss.GENII_test

dict_keys(['PV_6', 'PV_7'])